## Clustering template

### 1. Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm.notebook import tqdm

In [2]:
random.seed(42)
np.random.seed(42)
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
tqdm.pandas()
sns.set()

In [5]:
from helper_funcs import *

### 2. Reading data

In [ ]:
df = pd.read_csv('',index_col=0)

### 3. Exploring data

### 4. Feature engineering

### 5. Preparing X

In [ ]:
X = df[feats]

In [ ]:
X_hot = pd.get_dummies(X)

In [ ]:
# normalize
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [ ]:
# pca
from sklearn.decomposition import PCA

X_red = PCA(n_components=2).fit_transform(X_imp_std)

### 6. Models with default parameters

#### 6.1 Kmeans

In [ ]:
distortions = []
for i in tqdm(range(1, 11)):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(X_imp_std)
    distortions.append(km.inertia_)

# plot
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
# k means
from sklearn.cluster import KMeans

kmeans = KMeans(init='k-means++', n_clusters=3, n_init=10)
kmeans.fit(X_imp_std)

In [ ]:
y_pred = kmeans.predict(X_imp_std)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(X_red[:,0],X_red[:,1],hue=y_pred)

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(n_jobs=-1)
db.fit(X_imp_std)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(X_red[:,0],X_red[:,1],hue=db.labels_)

### 7. Grid search

#### 7.1 Random forest

In [ ]:
n_estimators = [1,5,10,50,100,200,500]
max_depth = [1,5,10,20,30,50]

max_features = ['auto', 'sqrt']
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

%%time
#neg_mean_absolute_error, neg_mean_squared_error
rf_grid = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf_grid, param_distributions=random_grid, scoring='neg_mean_absolute_error',n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)

rf_random.best_params_

rf_best = rf_random.best_estimator_
rf_best_scores = score_model(rf_best, X_train, y_train, X_test, y_test, name='best_rf')